## Step 1: Calculate emissions factors for different fuel sources

## Fossil Fuels Emissions Factors
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf

In [ ]:
# Calculate emissions factors for fossil fuels
# This is before adjusting for natural gas leakage
# Note: We use electricity marginal damages directly instead of multiplying
# CEDM emissions factors by the EASIUR marginal damages. 
def calculate_fossilFuel_emission_factor(fuel_type, so2_factor, nox_factor, pm25_factor, fuelConversion_factor1, fuelConversion_factor2):
    """
    Calculate Emissions Factors: FOSSIL FUELS
    Fossil Fuels (Natural Gas, Fuel Oil, Propane):
    - NOx, SO2, CO2: 
        - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
        - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
        - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
        - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
    - PM2.5: 
        - A National Methodology and Emission Inventory for Residential Fuel Combustion
        - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
    """
    
    # Create an empty dictionary called margEmis_factors to store the values
    margEmis_factors = {}

    # SO2, NOx, CO2: (_ lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
    # PM2.5 - FUEL OIL: 0.83 lb/thousand gallons * (1 thousand gallons / 1000 gallons) * (1 gallon heating oil/138,500 BTU) * (3412 BTU/1 kWh)
    # PM2.5 - NATURAL GAS: 1.9 lb/million cf * (million cf/1000000 cf) * (1 cf natural gas/1039 BTU) * (3412 BTU/1 kWh)
    # PM2.5 - PROPANE: 0.17 lb/thousand gallons * (1 thousand gallons / 1000 gallons) * (1 gallon propane/91,452 BTU) * (3412 BTU/1 kWh)
    margEmis_factors[f"{fuel_type}_so2"] = so2_factor * (1 / 1000000) * (3412 / 1)
    margEmis_factors[f"{fuel_type}_nox"] = nox_factor * (1 / 1000000) * (3412 / 1)
    margEmis_factors[f"{fuel_type}_pm25"] = pm25_factor * (1 / fuelConversion_factor1) * (1 / fuelConversion_factor2) * (3412 / 1)

    # NATURAL GAS LEAKAGE: NATURAL GAS INFRASTRUCTURE
    # leakage rate for natural gas infrastructure
    # 1 Therm = 29.30 kWh --> 1.27 kg CO2e/therm * (1 therm/29.30 kWh) = 0.043 kg CO2e/kWh = 0.095 lb CO2e/kWh
    naturalGas_leakage_mtCO2e_perkWh = 0.043 * (1/1000)

    # CO2e include pre- and post-combustion emissions
    margEmis_factors[f"naturalGas_co2e"] = (228.5 * (1/1000) * (1/1000)) + naturalGas_leakage_mtCO2e_perkWh
    margEmis_factors[f"propane_co2e"]  = 275.8 * (1/1000) * (1/1000)
    margEmis_factors[f"fuelOil_co2e"]  = 303.9 * (1/1000) * (1/1000)

    return margEmis_factors

print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
Fossil Fuels (Natural Gas, Fuel Oil, Propane):
- NOx, SO2, CO2: 
    - RESNET Table 7.1.2 Emissions Factors for Household Combustion Fuels
    - Source: https://www.resnet.us/wp-content/uploads/ANSIRESNETICC301-2022_resnetpblshd.pdf
    - All factors are in units of lb/Mbtu so energy consumption in kWh need to be converted to kWh 
    - (1 lb / Mbtu) * (1 Mbtu / 1x10^6 Btu) * (3412 Btu / 1 kWh)
- PM2.5: 
    - A National Methodology and Emission Inventory for Residential Fuel Combustion
    - Source: https://www3.epa.gov/ttnchie1/conference/ei12/area/haneke.pdf
-------------------------------------------------------------------------------------------------------
""")

fuelOil_factors = calculate_fossilFuel_emission_factor(fuel_type="fuelOil", so2_factor=0.0015, nox_factor=0.1300, pm25_factor=0.83, fuelConversion_factor1=1000, fuelConversion_factor2=138500)
naturalGas_factors = calculate_fossilFuel_emission_factor(fuel_type="naturalGas", so2_factor=0.0006, nox_factor=0.0922, pm25_factor=1.9, fuelConversion_factor1=1000000, fuelConversion_factor2=1039)
propane_factors = calculate_fossilFuel_emission_factor(fuel_type="propane", so2_factor=0.0002, nox_factor=0.1421, pm25_factor=0.17, fuelConversion_factor1=1000, fuelConversion_factor2=91452)

all_factors = {**fuelOil_factors, **naturalGas_factors, **propane_factors}

df_margEmis_factors = pd.DataFrame.from_dict(all_factors, orient="index", columns=["value"])
df_margEmis_factors.reset_index(inplace=True)
df_margEmis_factors.columns = ["pollutant", "value"]
df_margEmis_factors[["fuel_type", "pollutant"]] = df_margEmis_factors["pollutant"].str.split("_", expand=True)
# df_margEmis_factors["unit"] = "[lb/kWh]"

# Update the units to metric tons per kWh
df_margEmis_factors["unit"] = "[mt/kWh]"

# Convert the values from lb/kWh to mt/kWh
lb_to_mt = 0.00045359237
df_margEmis_factors["value"] = df_margEmis_factors["value"] * lb_to_mt

# Add the 'state' column and assign 'National' to every row
df_margEmis_factors = df_margEmis_factors.assign(state='National')

df_margEmis_factors = df_margEmis_factors[["state", "fuel_type", "pollutant", "value", "unit"]]
df_margEmis_factors

## Electricity Marginal Emissions Factors
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]"
- Predictor: Year"
- Pollutants: SO2, NOx, PM2.5, CO2"

In [ ]:
print("""
-------------------------------------------------------------------------------------------------------
Public Perspective: Monetized Marginal Damages from Emissions
-------------------------------------------------------------------------------------------------------
Step 1: Calculate emissions factors for different fuel sources
- Electricity
- Natural Gas
- Fuel Oil 
- Propane
-------------------------------------------------------------------------------------------------------
""")

In [ ]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: ELECTRICITY
-------------------------------------------------------------------------------------------------------
Electricity Marginal Emissions Factors:
- STATE Regional Aggregation is what is used in the Parth Analysis 
- "Marginal Emissions Factors for Electricity"
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Emissions [kg/MWh]
- Predictor: Year")
- Pollutants: SO2, NOx, PM2.5, CO2
-------------------------------------------------------------------------------------------------------
""")
filename = 'Generation-MARREG-EMIT-state-byYear.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margEmissions = pd.read_csv(file_path, index_col=0)

# Convert from kg/MWh to lb/kWh (kg/MWh) * (lb/kg) * (1 MWh)
# Obtain value from the CSV file and convert to lbs pollutant per kWh 
df_margEmis_electricity = pd.DataFrame({
    'state': df_margEmissions['region'],
    'fuel_type': 'electricity',
    'pollutant': df_margEmissions['pollutant'],
    'value': df_margEmissions['factor'] * (2.20462/1) * (1/1000),
    'unit': '[lb/kWh]'
})
df_margEmis_electricity

### Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage

In [ ]:
print("""
-------------------------------------------------------------------------------------------------------
Step 2: Adjust Natural Gas & Electricity Emissions Factors for Natural Gas Leakage
-------------------------------------------------------------------------------------------------------
Natural Gas (Deetjen et al.): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per therm of natural gas consumed for heating and 
convert to CO2-equivalent emissions via the GWP of methane. We assume that for every therm of 
natural gas consumed for heating, 0.023 therms of methane escape to the atmosphere [28]. 
Using the energy density of natural gas, we convert from therms to kilograms and multiply 
by 28—the GWP of methane [29]—to calculate a rate of 1.27 kg CO2-equivalent per therm of 
consumed natural gas."

Electricity NERC Regions (Deetjen et al): 
"To account for the natural gas infrastructure's leakage of the greenhouse gas methane, 
we estimate the amount of methane leaked per MWh of electricity generation in each NERC 
region and convert to CO2-equivalent emissions via the global warming potential (GWP) of methane. 
For example, we find that in 2017, the states comprising the western region (WECC) of 
the US electric grid consumed 1.45 million MMcf of natural gas in the power sector [27]. 
We assume that for every MMcf of consumed natural gas, 0.023 MMcf of methane is leaked into 
the atmosphere [28]. By multiplying that leakage rate by the 1.45 million MMcf of consumed 
natural gas, converting to tonnes, and multiplying by a GWP of 28 [29], we estimate 
that the 2017 WECC power sector contributed to methane leakage amounting to 18.6 Mt CO2-equivalent.
By dividing this 18.6 Mt by the 724 TWh of the WECC states' generated electricity [27], we 
calculate a methane leakage rate factor of 25.7 kg MWh−1. In the same manner, we calculate the 
methane leakage rate factors for the other NERC regions. We use the 100 years GWP value of 28 
for methane. Although there have been proposals to use 20 years GWP values, recent research 
shows that the benefits of this alternative 20 years time from are overstated [30]."
-------------------------------------------------------------------------------------------------------
""")
filename = 'natural_gas_leakage_rate.csv'
relative_path = os.path.join(r"margEmis_electricity", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_naturalGas_leakage_rate = pd.read_csv(file_path)

state_abbreviations = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

# Map full state names to abbreviations
df_naturalGas_leakage_rate['state'] = df_naturalGas_leakage_rate['state_name'].map(state_abbreviations)

# thousand Mcf * (0.023 Mcf leak/1 Mcf) * (19.3 tonnes/1000 Mcf) * (1000 kg/1 tonne) * (2.205 lb/1 kg)) / (thousand MWh * (1000 MWh/thousand MWh)) 
df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] = (df_naturalGas_leakage_rate['naturalGas_electricity_generation'] * (0.023/1) * (19.3/1) * (1000/1) * (2.205/1)) / (df_naturalGas_leakage_rate['net_generation'] * (1000/1)) 

# (lb CH4/MWh) * (28 lb CO2e/1 lb CH4)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCH4_perMWh'] * (28/1)

# (lb CO2e/MWh) * (1 MWh / 1000 kWh)
df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perkWh'] = df_naturalGas_leakage_rate['naturalGas_leakage_lbCO2e_perMWh'] * (1/1000)
df_naturalGas_leakage_rate

In [ ]:
# NATURAL GAS LEAKAGE: NATURAL GAS USED IN ELECTRICITY GENERATION
if 'naturalGas_leakage_lbCO2e_perkWh' in df_margEmis_electricity.columns:
    df_margEmis_electricity.drop(columns=['naturalGas_leakage_lbCO2e_perkWh'], inplace=True)

df_margEmis_electricity = df_margEmis_electricity.merge(
    df_naturalGas_leakage_rate[['state', 'naturalGas_leakage_lbCO2e_perkWh']],
    how='left',  # Use a left join to keep all rows from df_margEmis_electricity
    on=['state']  # Merge on the 'state' column
)
# Set 'naturalGas_leakage_lbCO2e_perkWh' to zero where 'pollutant' is not 'co2'
df_margEmis_electricity.loc[df_margEmis_electricity['pollutant'] != 'co2', 'naturalGas_leakage_lbCO2e_perkWh'] = 0.0

# Calculate adjusted marginal emissions factore with natural gas fugitive emissions
df_margEmis_electricity['margEmis_factor_adjusted'] = df_margEmis_electricity['value'] + df_margEmis_electricity['naturalGas_leakage_lbCO2e_perkWh'] 

# Create a factor to multiply marginal damages by
df_margEmis_electricity['naturalGas_leakage_factor'] = df_margEmis_electricity['margEmis_factor_adjusted'] / df_margEmis_electricity['value']

# Reorder columns
df_margEmis_electricity = df_margEmis_electricity[['state', 'fuel_type', 'pollutant', 'value', 'unit', 'naturalGas_leakage_lbCO2e_perkWh', 'margEmis_factor_adjusted', 'naturalGas_leakage_factor']]
df_margEmis_electricity

In [ ]:
# Append df_margEmissions_electricity to df_margEmis_factors
# This produces a dataframe of marginal emissions rates for various fuel types
df_margEmis_factors = pd.concat([df_margEmis_factors, df_margEmis_electricity], ignore_index=True)
df_margEmis_factors

### Step 3: Quantify monitized damages using EASIUR Marginal Social Cost Factors
#### THE STEPS BELOW SUMMARIZE WHAT WAS DONE TO OBTAIN ALL NATIONAL EASIUR VALUES INCLUDED ON GITHUB
- Obtain all of the dwelling unit latitude and longitude values from the metadata columns
- Make a new dataframe of just the longitude and latitude values 
    - Make sure that the order is (longitude, latitude)
    - Do not include the index or column name when exporting 
- Export the CSV
- **Upload csv to EASIUR Website:**
    - Website: https://barney.ce.cmu.edu/~jinhyok/easiur/online/
    - See inputs in respective sections
- Download the file and put it in the 'easiur_batchConversion_download' folder
- Copy and paste the name of the file EASIUR generated when prompted
- Copy and paste the name of the filepath for the 'easiur_batchConversion_download' folder when prompted
- Match up the longitude and latitudes for each dwelling unit with the selected damages

### Fossil Fuels: EASIUR Marginal Damage (Social Cost) Factors Info
- Factor Type: Marginal Social Cost
- Calculation Method: Regression
- Metric: Marginal Social Cost [USD per metric ton]
- Dollar Year: 2010
- Income Year: 2018
- Population Year: 2018
- Aggregation: Longitude, and Latitude Coordinates
- Pollutants: Primary PM2.5, Sulfur Dioxide (SO2), Nitrogen Oxides (NOx), Ammonia (NH3)
- Elevation (Ground, 150m, 300m) and Seasons (Winter, Spring, Summer, Fall)

In [ ]:
# Create a dataframe containing just the longitude and Latitude
df_EASIUR_batchConversion = pd.DataFrame({
    'Longitude':df_euss_am_baseline['in.weather_file_longitude'],
    'Latitude':df_euss_am_baseline['in.weather_file_latitude'],
})

# Drop duplicate rows based on 'Longitude' and 'Latitude' columns
df_EASIUR_batchConversion.drop_duplicates(subset=['Longitude', 'Latitude'], keep='first', inplace=True)

# Create a location ID for the name of the batch conversion file
while True:
    if menu_state == 'N':
        location_id = 'National'
        print("You chose to analyze all of the United States.")
        break
    elif menu_state == 'Y':
        if menu_city == 'N':
            try:
                location_id = str(input_state)
                print(f"Location ID is: {location_id}")
                break
            except ValueError:
                print("Invalid input for state!")
        elif menu_city == 'Y':
            try:
                location_id = input_cityFilter.replace(', ', '_').strip()
                print(f"Location ID is: {location_id}")
                break
            except AttributeError:
                print("Invalid input for city filter!")
        else:
            print("Incorrect state or city filter assignment!")
    else:
        print("Invalid data location. Check your inputs at the beginning of this notebook!")

# Updated GitHub code has EASIUR file with all unique latitude, longitude coordinates in the US
filename = 'easiur_National2024-06-1421-22.csv'
relative_path = os.path.join(r"margDamages_EASIUR\easiur_batchConversion_download", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")
print("\n")

df_margSocialCosts = pd.read_csv(file_path)

# Convert from kg/MWh to lb/kWh
# Obtain value from the CSV file and convert to lbs pollutant per kWh 

# Define df_margSocialCosts_EASIUR DataFrame first
df_margSocialCosts_EASIUR = pd.DataFrame({
    'Longitude': df_margSocialCosts['Longitude'],
    'Latitude': df_margSocialCosts['Latitude']
})
df_margSocialCosts_EASIUR

### Step 4: Inflate Marginal Social Cost (Damage) Factors using BLS CPI for All Urban Consumers (CPI-U)
- Series Id:	CUUR0000SA0
- Not Seasonally Adjusted
- Series Title:	All items in U.S. city average, all urban consumers, not seasonally adjusted
- Area:	U.S. city average
- Item:	All items
- Base Period:	1982-84=100

### Use the updated Social Cost of Carbon (190 USD-2020/ton CO2) and inflate to USD-2022
- EPA Median for 2% near term discount rate and most commonly mentioned value is 190 USD-2020 using the GIVE model.
- 190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
- 190 USD value and inflate to USD 2022 because there is a clear source and ease of replicability.

### Adjustment for VSL
- EASIUR uses a VSL of 8.8M USD-2010 
- New EPA VSL is 11.3M USD-2021
- INFLATE TO $USD-2022

### ALL DOLLAR VALUES ARE NOW IN USD2022, PREVIOUSLY USED $USD-2021

In [ ]:
# Load the BLS Inflation Data
filename = 'bls_cpiu_2005-2023.xlsx'
relative_path = os.path.join(r"inflation_data", filename)
file_path = os.path.join(project_root, relative_path)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Create a pandas dataframe
df_bls_cpiu = pd.read_excel(file_path, sheet_name='bls_cpiu')

df_bls_cpiu = pd.DataFrame({
    'year': df_bls_cpiu['Year'],
    'cpiu_annual': df_bls_cpiu['Annual']
})

# Obtain the Annual CPIU values for the years of interest
bls_cpi_annual_2008 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2008)].item()
bls_cpi_annual_2010 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2010)].item()
bls_cpi_annual_2013 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2013)].item()
bls_cpi_annual_2018 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2018)].item()
bls_cpi_annual_2019 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2019)].item()
bls_cpi_annual_2020 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2020)].item()
bls_cpi_annual_2021 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2021)].item()
bls_cpi_annual_2022 = df_bls_cpiu['cpiu_annual'].loc[(df_bls_cpiu['year'] == 2022)].item()

# Precompute constant values
cpi_ratio_2022_2022 = bls_cpi_annual_2022 / bls_cpi_annual_2022
cpi_ratio_2022_2021 = bls_cpi_annual_2022 / bls_cpi_annual_2021  # For EPA VSL (11.3M USD-2021)
cpi_ratio_2022_2020 = bls_cpi_annual_2022 / bls_cpi_annual_2020  # For SCC
cpi_ratio_2022_2019 = bls_cpi_annual_2022 / bls_cpi_annual_2019 
cpi_ratio_2022_2018 = bls_cpi_annual_2022 / bls_cpi_annual_2018 
cpi_ratio_2022_2013 = bls_cpi_annual_2022 / bls_cpi_annual_2013
cpi_ratio_2022_2010 = bls_cpi_annual_2022 / bls_cpi_annual_2010
cpi_ratio_2022_2008 = bls_cpi_annual_2022 / bls_cpi_annual_2008  # For EPA VSL and SCC

# 2021 US EPA VSL is $11.3M in 2021 
# INFLATE TO USD2022, PREVIOUSLY USD2021
df_margSocialCosts_EASIUR['current_VSL_USD2022'] = 11.3 * cpi_ratio_2022_2021

# Easiur uses a VSL of $8.8 M USD2010
# Inflate to 2022 $USD
# PREVIOUSLY USD2021
df_margSocialCosts_EASIUR['easiur_VSL_USD2022'] = 8.8 * cpi_ratio_2022_2010

# Use df_margSocialCosts_EASIUR in the calculation of other columns
# Also adjust the VSL
df_margSocialCosts_EASIUR['margSocialCosts_pm25'] = round((df_margSocialCosts['PM25 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2022']/df_margSocialCosts_EASIUR['easiur_VSL_USD2022'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_so2'] = round((df_margSocialCosts['SO2 Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2022']/df_margSocialCosts_EASIUR['easiur_VSL_USD2022'])), 2)
df_margSocialCosts_EASIUR['margSocialCosts_nox'] = round((df_margSocialCosts['NOX Annual Ground'] * (1/2204.6) * (df_margSocialCosts_EASIUR['current_VSL_USD2022']/df_margSocialCosts_EASIUR['easiur_VSL_USD2022'])), 2)

# Note that SCC of $190 USD-2020 has some inconsistency with the VSL being used. An old study and 2008 VSL is noted
# We use the $190 USD value and inflate to USD 2022 because there is a clear source and ease of replicability.
# PREVIOUSLY USED USD2021
df_margSocialCosts_EASIUR['margSocialCosts_co2'] = round((190 * cpi_ratio_2022_2020 * (1/2204.6)), 2)
df_margSocialCosts_EASIUR['unit'] = '[$USD2022/lb]'
df_margSocialCosts_EASIUR

## Electricity CEDM-EASIUR Marginal Damages: Current and Decarbonizing Grid
- Factor Type: Marginal
- Calculation Method: Regression
- Metric: Marginal Damages EASIUR [USD per MWh or kWh]
- Year: 2018
- Regional Aggregation: eGRID subregion (all regions)
- Pollutants: SO2, NOx, PM2.5 CO2

SCC Adjustment: We use the EPA suggested 190 USD-2020 value for the social cost of carbon and inflate to 2022 USD. **PREVIOUSLY USED 2021 USD**

VSL: "We use a value of a statistical life (VSL) of USD 8.8 million (in 2010 dollars) for both our AP2 and EASIUR calculations. EASIUR reports damage intensities in USD/metric ton using this VSL and dollar year."

Bistline et al. 2023 (ERL Paper): 
- ERL Paper: https://iopscience.iop.org/article/10.1088/1748-9326/ad0d3b/meta#erlad0d3bs6
- Data on Zenodo: https://zenodo.org/records/8322973
- Health related emissions reductions from 2021 levels (assumed mostly coal):
    - "In 2035, unabated coal reductions from 2021 levels range from 44%–100% with IRA (84% average) versus 12%–63% in the reference (38% average)."
    - Pre-IRA Scenario: -38% from 2021 levels
    - IRA-Ref Scenario: -84% from 2021 levels
    - IRA-High Scenario: -100% from 2021 levels
- CO2 reductions from 2005 (see image):
    - Pre-IRA Scenario: -53% from 2005 levels
    - IRA-Ref Scenario: -78% from 2005 levels
    - IRA-High Scenario: -87% from 2005 levels

![grid_decarb_assumptions.png](attachment:grid_decarb_assumptions.png)

In [ ]:
# For CO2 adjust SCC
# Create an adjustment factor for the new Social Cost of Carbon (SCC)
epa_scc = 190 * cpi_ratio_2022_2020
old_scc = 40 * cpi_ratio_2022_2010
scc_adjustment_factor = epa_scc / old_scc

# For Health-Related Emissions Adjust for different Value of a Statistical Life (VSL) values
# Current VSL is $11.3 M USD2021
# INFLATE TO USD2022, PREVIOUSLY USD2021
current_VSL_USD2022 = 11.3 * cpi_ratio_2022_2021

# Easiur uses a VSL of $8.8 M USD2010
# INFLATE TO USD2022, PREVIOUSLY USD2021
easiur_VSL_USD2022 = 8.8 * (cpi_ratio_2022_2010)

# Calculate VSL adjustment factor
vsl_adjustment_factor = current_VSL_USD2022 / easiur_VSL_USD2022

### Damages from Health Related Emissions

In [ ]:
filename = 'Generation-MARREG-DAMEASIUR-egrid-byYear_health2018.csv'
relative_path = os.path.join(r"margDamages_EASIUR", filename)
file_path = os.path.join(project_root, relative_path)
df_margDamages_health2018 = pd.read_csv(file_path, index_col=0)

print(f"Retrieved data for filename: {filename}")
print(f"Located at filepath: {file_path}")

# Marginal damages [$/kWh]
# Inflate from 2010 to 2023
# Note only 2018 data available, used in place of 2022 or 24.
df_margDamages_EASIUR_health = pd.DataFrame({
    'subregion_eGRID': df_margDamages_health2018['region'],
    'pollutant': df_margDamages_health2018['pollutant'],
    'unit': '[$USD2023/kWh]',
    'margDamages_dollarPerkWh_adjustVSL_2018': (df_margDamages_health2018['factor'] * (vsl_adjustment_factor) * (1/1000)) * (cpi_ratio_2023_2010)
})

df_margDamages_EASIUR_health['preIRA_margDamages_decarb_2035'] = 

df_margDamages_EASIUR_health['iraRef_margDamages_decarb_2035'] = 

# df_margDamages_EASIUR_health

In [ ]:
# Look at the coal generation under different scenarios create projection factors based off of coal generation starting in 

In [ ]:
# Combine them top to bottom
df_margDamages_EASIUR = pd.concat([df_margDamages_EASIUR_climate, df_margDamages_EASIUR_health], ignore_index=True)
df_margDamages_EASIUR

In [ ]:
# Marginal Damages for a Gradually Decarbonizing Grid
df_margDamages_gridDecarb = df_margDamages_EASIUR.copy()

# Assuming df_margDamages_gridDecarb is already a copy of df_margDamages_EASIUR
years = list(range(2018, 2051))
scenario_list = ['preIRA_', 'iraRef_', 'iraHigh_']

# Initialize columns for each policy_scenario and year based on 2018 reference
for policy_scenario in scenario_list:
    for year in years:
        # Define the name of the new column for the current year and policy_scenario
        column_name = f'{policy_scenario}margDamages_dollarPerkWh_adjustVSL_{year}'
        
        if year == 2018:
            # For the base year 2018, use the existing marginal damages as the starting point
            df_margDamages_gridDecarb[column_name] = df_margDamages_gridDecarb['margDamages_dollarPerkWh_adjustVSL_2018']
        else:
            # Initialize other years' columns to None, to be filled in later steps
            df_margDamages_gridDecarb[column_name] = None

# Apply reductions iteratively for each year and policy_scenario
for policy_scenario in scenario_list:
    for year in years:
        # Define the name of the current year's column and the previous year's column
        column_name = f'{policy_scenario}margDamages_dollarPerkWh_adjustVSL_{year}'
        previous_year_column = f'{policy_scenario}margDamages_dollarPerkWh_adjustVSL_{year-1}'

        # Handle reductions for years between 2019 and 2035
        if year > 2018 and year <= 2035:
            # Determine if the pollutant is CO2 (climate-related) for applying early reductions
            is_climate = df_margDamages_gridDecarb['pollutant'] == 'co2'
            
            # Determine if the year is 2022 or later, when health-related reductions begin
            is_health_period = year >= 2022

            # Retrieve the annual reduction factor for the current policy_scenario
            reduction = df_margDamages_gridDecarb[f'{policy_scenario}reduction_margDamages_annual']
            
            # Apply the reduction only if it's relevant:
            # - For CO2 (climate) reductions start from 2019 onwards.
            # - For health-related pollutants, reductions start from 2022 onwards.
            df_margDamages_gridDecarb[column_name] = df_margDamages_gridDecarb[previous_year_column] - reduction * (is_climate | is_health_period)

        # For years after 2035, no further reductions are applied
        elif year > 2035:
            # Set the value to be the same as the 2035 value, maintaining it constant
            df_margDamages_gridDecarb[column_name] = df_margDamages_gridDecarb[f'{policy_scenario}margDamages_dollarPerkWh_adjustVSL_2035']

# Ensure all values in the numeric columns are non-negative by clipping at zero
numeric_columns = df_margDamages_gridDecarb.select_dtypes(include=['float64', 'int64']).columns
df_margDamages_gridDecarb[numeric_columns] = df_margDamages_gridDecarb[numeric_columns].clip(lower=0)

# # Output the final DataFrame to check the calculated marginal damages
# print(df_margDamages_gridDecarb)

In [ ]:
# Create a lookup dictionary for the state-specific emissions factors for electricity
electricity_factors = df_margEmis_factors[df_margEmis_factors['fuel_type'] == 'electricity']
emis_electricity_lookup = {(row['pollutant'], row['state']): row['margEmis_factor_adjusted'] for _, row in electricity_factors.iterrows()}

# ELECTRICITY DAMAGES LOOKUP: Previously damages_CEDM_lookup
damages_CEDM_lookup = {(row['pollutant'], row['subregion_eGRID']): row['margDamages_dollarPerkWh_adjustVSL_ref'] for _, row in df_margDamages_EASIUR.iterrows()}

# GRID policy_scenario-SPECIFIC EMISSIONS FACTORS!!!
# Create empty dictionaries to store the lookup data for each policy_scenario
preIRA_damages_electricity_lookup = {}
iraRef_damages_electricity_lookup = {}
iraHigh_damages_electricity_lookup = {}

# Assuming df_margDamages_gridDecarb is already a copy of df_margDamages_EASIUR
years = list(range(2018, 2051))
scenario_list = ['preIRA_', 'iraRef_', 'iraHigh_']

# Define a mapping for policy_scenario descriptions
scenario_description_map = {
    'preIRA_': 'No Inflation Reduction Act',
    'iraRef_': 'AEO2023 Reference Case',
    'iraHigh_': 'High Uptake of Inflation Reduction Act'
}

for policy_scenario in scenario_list:
    for year in years:
        # Create an empty dictionary for the current year
        year_lookup = {}

        for _, row in df_margDamages_gridDecarb.iterrows():
            # Include the policy_scenario description in the key
            key = (row['pollutant'], row['subregion_eGRID'], scenario_description_map[policy_scenario])

            if policy_scenario == 'preIRA_':
                year_lookup[key] = row[f'preIRA_margDamages_dollarPerkWh_adjustVSL_{year}']
                preIRA_damages_electricity_lookup[year] = year_lookup

            elif policy_scenario == 'iraRef_':
                year_lookup[key] = row[f'iraRef_margDamages_dollarPerkWh_adjustVSL_{year}']
                iraRef_damages_electricity_lookup[year] = year_lookup

            elif policy_scenario == 'iraHigh_':
                year_lookup[key] = row[f'iraHigh_margDamages_dollarPerkWh_adjustVSL_{year}']
                iraHigh_damages_electricity_lookup[year] = year_lookup

# Now, you have three dictionaries for lookup:
# preIRA_damages_electricity_lookup, iraRef_damages_electricity_lookup, iraHigh_damages_electricity_lookup
# iraRef_damages_electricity_lookup

In [ ]:
print("""
-------------------------------------------------------------------------------------------------------
Calculate Emissions Factors: FOSSIL FUELS
-------------------------------------------------------------------------------------------------------
""")
pollutants = ['so2', 'nox', 'pm25', 'co2']

# Create a lookup dictionary of emissions factors for fossil fuels
fossilFuel_factors = df_margEmis_factors[df_margEmis_factors['state'] == 'National']
emis_fossilFuel_lookup = {(row['fuel_type'], row['pollutant']): row['margEmis_factor_adjusted'] for _, row in fossilFuel_factors.iterrows()}

# FOSSIL FUELS DAMAGES LOOKUP
# Create a damages_fossilFuel_lookup dictionary from df_margSocialCosts_EASIUR
damages_fossilFuel_lookup = df_margSocialCosts_EASIUR.groupby(['Longitude', 'Latitude']).first().to_dict()

### Step 5: Calculate End-use specific marginal damages
**I used the total emissions column for each of the end uses for the following reasons:**
- Most homes only have 1 of each end-use, so it is unlikely that the homes have a significant consumption values from different fuel types. Thus, the total consumption and total emissions column (sum of each dwelling units consumption by end-use for each fuel) is fine to use to calculate marginal damages (social cost)
- We can visualize the emissions in 2 by 2 grid (CO2, PM25, SO2, NOx) with each appliance's heating fuel in a different shape or color. 

### Baseline Marginal Damages: WHOLE-HOME

In [ ]:
print("""
-------------------------------------------------------------------------------------------------------
Step 5: Calculate End-use specific marginal damages
-------------------------------------------------------------------------------------------------------
      
-------------------------------------------------------------------------------------------------------
Baseline Marginal Damages: WHOLE-HOME
-------------------------------------------------------------------------------------------------------
""")

# calculate_marginal_damages(df, menu_mp, policy_scenario)
df_euss_am_baseline_home = calculate_marginal_damages(df=df_euss_am_baseline_home,
                                                      menu_mp=menu_mp,
                                                      policy_scenario='No Inflation Reduction Act',
                                                      drop_pollutant_damage_cols=True
                                                     )
df_euss_am_baseline_home